In [57]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


In [58]:
import os
import pandas as pd

DATA_DIR = r"D:\UWE CLASS\CSCT master\Master_project\Data"   
OUT_TABLES = r"D:\UWE CLASS\CSCT master\Master_project\outputs\tables"
OUT_FIGS   = r"D:\UWE CLASS\CSCT master\Master_project\outputs\figures"

os.makedirs(OUT_TABLES, exist_ok=True)
os.makedirs(OUT_FIGS, exist_ok=True)


files = sorted(os.listdir(DATA_DIR))
files[:50], len(files)


(['RoadtransportEFs_NAEI23_v1.xlsx',
  'ghg-conversion-factors-2024-FlatFormat_v1_1.xlsx',
  'panel_activity_long.csv',
  'region_traffic_by_vehicle_type.csv',
  'region_year_co2e.csv',
  'region_year_panel_for_regression.csv',
  'region_year_vehicle_co2e_breakdown.csv',
  'tra0101-miles-by-vehicle-type.ods',
  'tra0101-miles-by-vehicle-type.xlsx',
  'traffic_2015_2024.csv'],
 10)

In [59]:
import os
import pandas as pd

DATA_DIR = r"D:\UWE CLASS\CSCT master\Master_project\Data"

cands = [f for f in os.listdir(DATA_DIR) if f.lower().startswith("region_year_co2e")]
print("Candidates:", cands)

if len(cands) == 0:
    raise FileNotFoundError("No file starting with 'region_year_co2e' found in DATA_DIR")

fname = cands[0]
DATA_FILE = os.path.join(DATA_DIR, fname)
print("Loading:", DATA_FILE)

if fname.lower().endswith(".csv"):
    df = pd.read_csv(DATA_FILE)
elif fname.lower().endswith((".xlsx", ".xls")):
    df = pd.read_excel(DATA_FILE)
else:
    raise ValueError(f"Unsupported file type: {fname}")

df.head(), df.shape




Candidates: ['region_year_co2e.csv']
Loading: D:\UWE CLASS\CSCT master\Master_project\Data\region_year_co2e.csv


(     region_name  year  co2e_tonnes_total
 0  East Midlands  2015       9.237719e+06
 1  East Midlands  2016       9.515900e+06
 2  East Midlands  2017       9.671759e+06
 3  East Midlands  2018       9.750602e+06
 4  East Midlands  2019       9.921433e+06,
 (110, 3))

In [60]:

df.columns


Index(['region_name', 'year', 'co2e_tonnes_total'], dtype='object')

In [61]:
import os
import pandas as pd

DATA_DIR = r"D:\UWE CLASS\CSCT master\Master_project\Data"

def load_any(path):
    if path.lower().endswith(".csv"):
        return pd.read_csv(path)
    elif path.lower().endswith((".xlsx", ".xls")):
        return pd.read_excel(path)
    else:
        raise ValueError(f"Unsupported file: {path}")

# file patterns ที่น่าจะเป็น fleet composition / activity
patterns = [
    "region_year_vehicle",
    "region_traffic_by_vehicle_type",
    "traffic_2015_2024",
    "panel_activity_long"
]

candidates = []
for f in os.listdir(DATA_DIR):
    fl = f.lower()
    if any(p in fl for p in patterns):
        candidates.append(f)

print("Candidate files:")
for f in candidates:
    print(" -", f)

# Preview columns for each candidate
previews = []
for f in candidates:
    p = os.path.join(DATA_DIR, f)
    try:
        tmp = load_any(p)
        previews.append((f, tmp.shape, list(tmp.columns)[:30]))
    except Exception as e:
        previews.append((f, "ERROR", str(e)))

previews


Candidate files:
 - panel_activity_long.csv
 - region_traffic_by_vehicle_type.csv
 - region_year_vehicle_co2e_breakdown.csv
 - traffic_2015_2024.csv


[('panel_activity_long.csv',
  (550, 8),
  ['year',
   'region_name',
   'region_ons_code',
   'vehicle_type',
   'activity_miles',
   'activity_km',
   'total_km',
   'share_activity']),
 ('region_traffic_by_vehicle_type.csv',
  (352, 13),
  ['year',
   'region_id',
   'region_name',
   'region_ons_code',
   'link_length_km',
   'link_length_miles',
   'pedal_cycles',
   'two_wheeled_motor_vehicles',
   'cars_and_taxis',
   'buses_and_coaches',
   'LGVs',
   'all_HGVs',
   'all_motor_vehicles']),
 ('region_year_vehicle_co2e_breakdown.csv',
  (550, 6),
  ['region_name',
   'year',
   'vehicle_std',
   'activity_km',
   'share_activity',
   'co2e_tonnes']),
 ('traffic_2015_2024.csv',
  (110, 13),
  ['year',
   'region_id',
   'region_name',
   'region_ons_code',
   'link_length_km',
   'link_length_miles',
   'pedal_cycles',
   'two_wheeled_motor_vehicles',
   'cars_and_taxis',
   'buses_and_coaches',
   'LGVs',
   'all_HGVs',
   'all_motor_vehicles'])]

In [62]:
import re

def score_fleet_df(df):
    cols = [c.lower() for c in df.columns]
    
    has_region = any("region" in c for c in cols) or any("region_name" in c for c in cols)
    has_year = any(c == "year" or "year" in c for c in cols)
    if not (has_region and has_year):
        return -1
    
    
    keywords = ["car", "lgv", "hgv", "bus", "coach", "motorcycle", "van", "taxi"]
    kw_count = sum(any(k in c for k in keywords) for c in cols)
    return kw_count

fleet_best = None
best_score = -1
fleet_best_name = None

for f in candidates:
    p = os.path.join(DATA_DIR, f)
    tmp = load_any(p)
    sc = score_fleet_df(tmp)
    if sc > best_score:
        best_score = sc
        fleet_best = tmp
        fleet_best_name = f

print("Best fleet/activity candidate:", fleet_best_name)
print("Score:", best_score)
print("Shape:", fleet_best.shape)
fleet_best.head()


Best fleet/activity candidate: region_traffic_by_vehicle_type.csv
Score: 4
Shape: (352, 13)


,year,region_id,region_name,region_ons_code,link_length_km,link_length_miles,pedal_cycles,two_wheeled_motor_vehicles,cars_and_taxis,buses_and_coaches,LGVs,all_HGVs,all_motor_vehicles
0,1993,1,South West,E12000009,48751.55,30292.81,228000000.0,289900000.0,1.967430e+10,245000000.0,2.351700e+09,1.295600e+09,2.385660e+10
1,1993,2,East Midlands,E12000004,30681.75,19064.76,199700000.0,172800000.0,1.620020e+10,190100000.0,2.082200e+09,1.647100e+09,2.029240e+10
2,1993,3,Scotland,S92000003,58532.65,36370.50,147300000.0,126400000.0,1.767760e+10,334300000.0,2.231000e+09,1.339900e+09,2.170930e+10
3,1993,4,Wales,W92000004,32543.50,20221.59,81700000.0,104600000.0,1.127880e+10,156300000.0,1.452700e+09,7.352000e+08,1.372760e+10
4,1993,5,North West,E12000002,35952.60,22339.91,251400000.0,199200000.0,2.373000e+10,339800000.0,2.801800e+09,1.834400e+09,2.890520e+10


In [63]:
fleet = fleet_best.copy()


dup_cols = fleet.columns[fleet.columns.duplicated()].tolist()
print("Duplicated columns:", dup_cols)


def dedup_columns(cols):
    seen = {}
    new_cols = []
    for c in cols:
        if c not in seen:
            seen[c] = 0
            new_cols.append(c)
        else:
            seen[c] += 1
            new_cols.append(f"{c}_{seen[c]+1}")
    return new_cols

fleet.columns = dedup_columns(list(fleet.columns))
print("After dedup, duplicated?:", fleet.columns.duplicated().any())
fleet.head()


Duplicated columns: []
After dedup, duplicated?: False


,year,region_id,region_name,region_ons_code,link_length_km,link_length_miles,pedal_cycles,two_wheeled_motor_vehicles,cars_and_taxis,buses_and_coaches,LGVs,all_HGVs,all_motor_vehicles
0,1993,1,South West,E12000009,48751.55,30292.81,228000000.0,289900000.0,1.967430e+10,245000000.0,2.351700e+09,1.295600e+09,2.385660e+10
1,1993,2,East Midlands,E12000004,30681.75,19064.76,199700000.0,172800000.0,1.620020e+10,190100000.0,2.082200e+09,1.647100e+09,2.029240e+10
2,1993,3,Scotland,S92000003,58532.65,36370.50,147300000.0,126400000.0,1.767760e+10,334300000.0,2.231000e+09,1.339900e+09,2.170930e+10
3,1993,4,Wales,W92000004,32543.50,20221.59,81700000.0,104600000.0,1.127880e+10,156300000.0,1.452700e+09,7.352000e+08,1.372760e+10
4,1993,5,North West,E12000002,35952.60,22339.91,251400000.0,199200000.0,2.373000e+10,339800000.0,2.801800e+09,1.834400e+09,2.890520e+10


In [64]:
emiss = df.copy()
emiss["year"] = emiss["year"].astype(int)


mapping = (
    fleet[["region_name", "region_ons_code"]]
    .dropna()
    .drop_duplicates()
)


emiss2 = emiss.merge(mapping, on="region_name", how="left")

print("Missing region_ons_code in emissions:", emiss2["region_ons_code"].isna().sum())
emiss2.head()


Missing region_ons_code in emissions: 0


,region_name,year,co2e_tonnes_total,region_ons_code
0,East Midlands,2015,9.237719e+06,E12000004
1,East Midlands,2016,9.515900e+06,E12000004
2,East Midlands,2017,9.671759e+06,E12000004
3,East Midlands,2018,9.750602e+06,E12000004
4,East Midlands,2019,9.921433e+06,E12000004


In [65]:
import numpy as np

# traffic (fleet) years
fleet["year"] = fleet["year"].astype(int)

fleet_1524 = fleet[(fleet["year"] >= 2015) & (fleet["year"] <= 2024)].copy()
print("fleet_1524 shape:", fleet_1524.shape)
print("fleet_1524 years:", (fleet_1524["year"].min(), fleet_1524["year"].max()))

panel = emiss2.merge(
    fleet_1524,
    on=["region_ons_code", "year"],
    how="inner",
    suffixes=("", "_traffic")
)

print("Merged panel shape:", panel.shape)
panel[["region_name", "region_ons_code", "year", "co2e_tonnes_total" if "co2e_tonnes_total" in panel.columns else "co2e_tonnes_total"]].head()


fleet_1524 shape: (110, 13)
fleet_1524 years: (np.int64(2015), np.int64(2024))
Merged panel shape: (110, 15)


,region_name,region_ons_code,year,co2e_tonnes_total
0,East Midlands,E12000004,2015,9.237719e+06
1,East Midlands,E12000004,2016,9.515900e+06
2,East Midlands,E12000004,2017,9.671759e+06
3,East Midlands,E12000004,2018,9.750602e+06
4,East Midlands,E12000004,2019,9.921433e+06


In [66]:
veh_cols = [
    "cars_and_taxis",
    "LGVs",
    "all_HGVs",
    "buses_and_coaches",
    "two_wheeled_motor_vehicles"
]

missing = [c for c in veh_cols if c not in panel.columns]
print("Missing vehicle cols:", missing)

panel["total_veh_activity"] = panel[veh_cols].sum(axis=1)

for c in veh_cols:
    panel[f"share_{c}"] = panel[c] / panel["total_veh_activity"].replace(0, np.nan)

# optional: log control
panel["ln_vehicle_activity"] = np.log(panel["total_veh_activity"].clip(lower=1e-9))

panel[[ "region_name","year","co2_tonnes_total" if "co2_tonnes_total" in panel.columns else "co2e_tonnes_total",
       "total_veh_activity"] + [f"share_{c}" for c in veh_cols]].head()


Missing vehicle cols: []


,region_name,year,co2e_tonnes_total,total_veh_activity,share_cars_and_taxis,share_LGVs,share_all_HGVs,share_buses_and_coaches,share_two_wheeled_motor_vehicles
0,East Midlands,2015,9.237719e+06,2.742470e+10,0.767199,0.149030,0.069317,0.006713,0.007741
1,East Midlands,2016,9.515900e+06,2.819000e+10,0.767662,0.148336,0.069365,0.007765,0.006871
2,East Midlands,2017,9.671759e+06,2.866560e+10,0.767565,0.148872,0.068884,0.008055,0.006625
3,East Midlands,2018,9.750602e+06,2.891750e+10,0.766702,0.149349,0.068875,0.007798,0.007276
4,East Midlands,2019,9.921433e+06,2.948880e+10,0.768339,0.149521,0.068002,0.007583,0.006555


In [67]:
OUT_PANEL = os.path.join(DATA_DIR, "region_year_panel_for_regression.csv")
panel.to_csv(OUT_PANEL, index=False)
print("Saved:", OUT_PANEL)
print("Final panel columns (subset):")
display(panel.columns)


Saved: D:\UWE CLASS\CSCT master\Master_project\Data\region_year_panel_for_regression.csv
Final panel columns (subset):


Index(['region_name', 'year', 'co2e_tonnes_total', 'region_ons_code',
       'region_id', 'region_name_traffic', 'link_length_km',
       'link_length_miles', 'pedal_cycles', 'two_wheeled_motor_vehicles',
       'cars_and_taxis', 'buses_and_coaches', 'LGVs', 'all_HGVs',
       'all_motor_vehicles', 'total_veh_activity', 'share_cars_and_taxis',
       'share_LGVs', 'share_all_HGVs', 'share_buses_and_coaches',
       'share_two_wheeled_motor_vehicles', 'ln_vehicle_activity'],
      dtype='object')

In [68]:
import numpy as np
import statsmodels.formula.api as smf

# dependent
panel["ln_co2"] = np.log(panel["co2e_tonnes_total"].clip(lower=1e-9))

# baseline: share_cars_and_taxis (omit)
# NOTE: Do NOT include baseline share to avoid collinearity (shares sum to ~1)
formula = """
ln_co2
~ share_LGVs
+ share_all_HGVs
+ share_buses_and_coaches
+ share_two_wheeled_motor_vehicles
+ ln_vehicle_activity
+ C(region_name)
+ C(year)
"""

# OLS with clustered SE by region
m = smf.ols(formula, data=panel).fit(
    cov_type="cluster",
    cov_kwds={"groups": panel["region_name"]}
)

print(m.summary())


                            OLS Regression Results                            
Dep. Variable:                 ln_co2   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.105e+05
Date:                Thu, 08 Jan 2026   Prob (F-statistic):           3.63e-27
Time:                        11:45:13   Log-Likelihood:                 762.23
No. Observations:                 110   AIC:                            -1474.
Df Residuals:                      85   BIC:                            -1407.
Df Model:                          24                                         
Covariance Type:              cluster                                         
                                                 coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

In [69]:
import pandas as pd

coef_table = pd.DataFrame({
    "coef": m.params,
    "std_err": m.bse,
    "t": m.tvalues,
    "p": m.pvalues
})

main_vars = [
    "share_LGVs",
    "share_all_HGVs",
    "share_buses_and_coaches",
    "share_two_wheeled_motor_vehicles",
    "ln_vehicle_activity"
]

coef_table.loc[[v for v in main_vars if v in coef_table.index]].sort_index()


,coef,std_err,t,p
ln_vehicle_activity,0.991846,0.003611,274.664747,0.000000e+00
share_LGVs,0.157537,0.014676,10.734572,7.004279e-27
share_all_HGVs,2.357404,0.030772,76.608199,0.000000e+00
share_buses_and_coaches,1.758470,0.068508,25.667954,2.665417e-145
share_two_wheeled_motor_vehicles,-0.461411,0.020065,-22.996312,5.074747e-117


In [70]:
formula_tot_nocontrol = """
ln_co2
~ share_LGVs
+ share_all_HGVs
+ share_buses_and_coaches
+ share_two_wheeled_motor_vehicles
+ C(region_name)
+ C(year)
"""

m2 = smf.ols(formula_tot_nocontrol, data=panel).fit(
    cov_type="cluster",
    cov_kwds={"groups": panel["region_name"]}
)

print(m2.summary())


                            OLS Regression Results                            
Dep. Variable:                 ln_co2   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                     393.8
Date:                Thu, 08 Jan 2026   Prob (F-statistic):           1.30e-11
Time:                        11:45:13   Log-Likelihood:                 329.99
No. Observations:                 110   AIC:                            -612.0
Df Residuals:                      86   BIC:                            -547.2
Df Model:                          23                                         
Covariance Type:              cluster                                         
                                                 coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

In [71]:
import numpy as np
import statsmodels.formula.api as smf

# 1) Create emissions intensity (CO2 per total vehicle activity)
panel["ln_intensity"] = np.log((panel["co2e_tonnes_total"] / panel["total_veh_activity"]).clip(lower=1e-12))

# 2) Two-way fixed effects via dummies + clustered SE by region
formula_int = """
ln_intensity
~ share_LGVs
+ share_all_HGVs
+ share_buses_and_coaches
+ share_two_wheeled_motor_vehicles
+ C(region_name)
+ C(year)
"""

m_int = smf.ols(formula_int, data=panel).fit(
    cov_type="cluster",
    cov_kwds={"groups": panel["region_name"]}
)

print(m_int.summary())


                            OLS Regression Results                            
Dep. Variable:           ln_intensity   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.451e+05
Date:                Thu, 08 Jan 2026   Prob (F-statistic):           1.96e-24
Time:                        11:45:13   Log-Likelihood:                 753.36
No. Observations:                 110   AIC:                            -1459.
Df Residuals:                      86   BIC:                            -1394.
Df Model:                          23                                         
Covariance Type:              cluster                                         
                                                 coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

In [72]:
import pandas as pd

coef_int = pd.DataFrame({
    "coef": m_int.params,
    "std_err": m_int.bse,
    "t": m_int.tvalues,
    "p": m_int.pvalues
})

main_vars = [
    "share_LGVs",
    "share_all_HGVs",
    "share_buses_and_coaches",
    "share_two_wheeled_motor_vehicles"
]

coef_int.loc[[v for v in main_vars if v in coef_int.index]].sort_index()


,coef,std_err,t,p
share_LGVs,0.168763,0.017635,9.569553,1.073687e-21
share_all_HGVs,2.386055,0.033660,70.887503,0.000000e+00
share_buses_and_coaches,1.751576,0.078634,22.274905,6.471504e-110
share_two_wheeled_motor_vehicles,-0.428405,0.026115,-16.404740,1.768818e-60
